<a href="https://colab.research.google.com/github/254francis/Intro_to_AI_Labs/blob/main/last_trial_assignment_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import heapq
from collections import deque

class Environment:
    def __init__(self):
        self.graph = {}  # Adjacency list
        self.heuristics = {}  # Heuristic values per node

    def add_edge(self, from_point, to_point, cost):
        self.graph.setdefault(from_point, []).append((to_point, cost))
        self.graph.setdefault(to_point, []).append((from_point, cost))  # If undirected

    def set_heuristic(self, point, value):
        self.heuristics[point] = value

class PathFinder:
    def __init__(self, environment):
        self.env = environment

    def fewest_stopovers(self, start, goal):
        visited = set([start])
        queue = deque([(start, [start])])

        while queue:
            node, path = queue.popleft()
            if node == goal:
                return path
            for neighbor, _ in self.env.graph.get(node, []):
                if neighbor not in visited:
                    visited.add(neighbor)
                    queue.append((neighbor, path + [neighbor]))
        return None

    def all_paths(self, start, goal):
        def dfs(node, path, visited, results):
            visited.add(node)
            path.append(node)

            if node == goal:
                results.append(path[:])
            else:
                for neighbor, _ in self.env.graph.get(node, []):
                    if neighbor not in visited:
                        dfs(neighbor, path, visited, results)

            path.pop()
            visited.remove(node)

        results = []
        dfs(start, [], set(), results)
        return results

    def uniform_cost_search(self, start, goal):
        pq = [(0, start, [])]
        visited = set()

        while pq:
            cost, node, path = heapq.heappop(pq)
            if node == goal:
                return cost, path + [node]
            if node not in visited:
                visited.add(node)
                for neighbor, w in self.env.graph[node]:
                    if neighbor not in visited:
                        heapq.heappush(pq, (cost + w, neighbor, path + [node]))
        return float('inf'), []

    def a_star(self, start, goal):
        pq = [(self.env.heuristics.get(start, 0), 0, start, [])]
        visited = set()

        while pq:
            est_total, cost_so_far, node, path = heapq.heappop(pq)
            if node == goal:
                return cost_so_far, path + [node]
            if node not in visited:
                visited.add(node)
                for neighbor, w in self.env.graph.get(node, []):
                    if neighbor not in visited:
                        g = cost_so_far + w
                        h = self.env.heuristics.get(neighbor, 0)
                        f = g + h
                        heapq.heappush(pq, (f, g, neighbor, path + [node]))
        return float('inf'), []

class Agent:
    def __init__(self, pathfinder):
        self.pathfinder = pathfinder

    def act(self, mode, start, goal):
        if mode == 'quick-response':
            return self.pathfinder.fewest_stopovers(start, goal)
        elif mode == 'explore':
            return self.pathfinder.all_paths(start, goal)
        elif mode == 'low-battery':
            return self.pathfinder.uniform_cost_search(start, goal)
        elif mode == 'heuristic-optimized':
            return self.pathfinder.a_star(start, goal)
        else:
            raise ValueError("Invalid mode.")

# ─── SETUP EXAMPLE ─

if __name__ == "__main__":
    env = Environment()

    # Define scan-point edges
    env.add_edge("S1", "S2", 2)
    env.add_edge("S1", "S3", 5)
    env.add_edge("S2", "S3", 1)
    env.add_edge("S2", "S4", 4)
    env.add_edge("S3", "S4", 1)

    # Manual heuristic values (lower is better; 0 for goal)
    env.set_heuristic("S1", 5)
    env.set_heuristic("S2", 3)
    env.set_heuristic("S3", 2)
    env.set_heuristic("S4", 0)

    pathfinder = PathFinder(env)
    agent = Agent(pathfinder)

    # Choose mode
    print("Modes: quick-response | explore | low-battery | heuristic-optimized")
    mode = input("Enter agent mode: ").strip()
    start = input("Enter start scan point (e.g., S1): ").strip().upper()
    goal = input("Enter goal scan point (e.g., S4): ").strip().upper()

    result = agent.act(mode, start, goal)

    if isinstance(result, tuple):
        cost, path = result
        print(f"Path: {path} | Total Cost: {cost}")
    else:
        print("Path:", result)


Modes: quick-response | explore | low-battery | heuristic-optimized
Enter agent mode: explore
Enter start scan point (e.g., S1): s1
Enter goal scan point (e.g., S4): s4
Path: [['S1', 'S2', 'S3', 'S4'], ['S1', 'S2', 'S4'], ['S1', 'S3', 'S2', 'S4'], ['S1', 'S3', 'S4']]
